In [ ]:
import numpy as np
from implementations import least_squares, ridge_regression, logistic_regression, reg_logistic_regression, sigmoid,mean_squared_error_gd, mean_squared_error_sgd

# Load the dataset
x_train = np.loadtxt('data/dataset/x_train_filtered.csv', delimiter=',', skiprows=1, dtype=float)
y_train = np.genfromtxt('data/dataset/y_train.csv', delimiter=',', skip_header=1, dtype=int, usecols=1)
x_test = np.loadtxt('data/dataset/x_test_filtered.csv', delimiter=',', skiprows=1, dtype=float)
y_train[np.where(y_train == -1)] = 0

print("x_train shape:", x_train.shape)
print("x_test shape:", x_test.shape)
print("y_train shape:", y_train.shape)

In [ ]:
### Validation set creation (80% train, 20% validation)

# Function to shuffle and split data into train and validation sets
def split_data(x, y, train_ratio=0.8, seed=1):
    np.random.seed(seed)
    indices = np.arange(x.shape[0])
    np.random.shuffle(indices)
    x_shuffled = x[indices]
    y_shuffled = y[indices]

    split_index = int(train_ratio * x.shape[0])
    x_train_split = x_shuffled[:split_index]
    y_train_split = y_shuffled[:split_index]
    x_val_split = x_shuffled[split_index:]
    y_val_split = y_shuffled[split_index:]

    return x_train_split, y_train_split, x_val_split, y_val_split

# Split the data into 80% train and 20% validation
x_train_split, y_train_split, x_val_split, y_val_split = split_data(x_train, y_train, train_ratio=0.8)

print("x_train_split shape:", x_train_split.shape)
print("y_train_split shape:", y_train_split.shape)
print("x_val_split shape:", x_val_split.shape)
print("y_val_split shape:", y_val_split.shape)